In [1]:
import os
from pathlib import Path

print("CWD:", os.getcwd())
print("models exists:", Path("models").exists())
print("model exists:", Path("models/hand_landmarker.task").exists())
print("alt model exists:", Path("../models/hand_landmarker.task").exists())

CWD: /Users/luchito/Desktop/sign-language-project/notebooks
models exists: False
model exists: False
alt model exists: True


In [7]:
import sys
sys.executable

'/Users/luchito/Desktop/sign-language-project/.venv/bin/python'

In [1]:
import sys
import cv2
import mediapipe as mp
import torch
import numpy as np

print("Python:", sys.version)
print("OpenCV:", cv2.__version__)
print("MediaPipe:", mp.__version__)
print("Torch:", torch.__version__)

print("MPS built:", torch.backends.mps.is_built())
print("MPS available:", torch.backends.mps.is_available())

Python: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:07:17) [Clang 14.0.6 ]
OpenCV: 4.12.0
MediaPipe: 0.10.31
Torch: 2.9.1
MPS built: True
MPS available: True


In [5]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Device:", device)

a = torch.randn(512, 512, device=device)
b = torch.randn(512, 512, device=device)
c = a @ b
print("matmul ok:", c.shape, c.device)

Device: mps
matmul ok: torch.Size([512, 512]) mps:0


In [ ]:
import time
import cv2
import mediapipe as mp
from pathlib import Path

# --- resolve project root (where .git lives), so the notebook can run from anywhere ---
root = Path.cwd()
while root != root.parent and not (root / ".git").exists():
    root = root.parent

MODEL_PATH = root / "models" / "hand_landmarker.task"
print("Resolved MODEL_PATH:", MODEL_PATH)

if not MODEL_PATH.exists():
    raise FileNotFoundError(
        f"Model file not found: {MODEL_PATH}\n"
        "Скачай модель в корень репозитория:\n"
        "mkdir -p models\n"
        "curl -L -o models/hand_landmarker.task "
        "https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/latest/hand_landmarker.task"
    )

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=str(MODEL_PATH)),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=2,
)

WIN = "MediaPipe HandLandmarker (q/ESC to quit)"
cv2.namedWindow(WIN, cv2.WINDOW_NORMAL)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Камера не открылась. Попробуй VideoCapture(1).")

start = time.time()

try:
    with HandLandmarker.create_from_options(options) as landmarker:
        while True:
            ok, frame = cap.read()
            if not ok:
                print("Не удалось прочитать кадр с камеры")
                break

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

            timestamp_ms = int((time.time() - start) * 1000)
            result = landmarker.detect_for_video(mp_image, timestamp_ms)

            if result.hand_landmarks:
                h, w = frame.shape[:2]
                for hand in result.hand_landmarks:
                    for lm in hand:
                        x, y = int(lm.x * w), int(lm.y * h)
                        cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

            cv2.imshow(WIN, frame)

            # если окно закрыли крестиком — выходим до следующего imshow
            if cv2.getWindowProperty(WIN, cv2.WND_PROP_VISIBLE) < 1:
                break

            key = cv2.waitKey(1) & 0xFF
            if key == ord("q") or key == 27:
                break
finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Camera test done")

Resolved MODEL_PATH: /Users/luchito/Desktop/sign-language-project/models/hand_landmarker.task


I0000 00:00:1767956801.132633 1736951 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2 Max
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1767956801.136853 1736953 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767956801.140638 1736958 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767956801.157704 1736954 landmark_projection_calculator.cc:81] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Camera test done


: 